<a href="https://colab.research.google.com/github/kirankamatmgm/steering-wheel-prediction/blob/master/Copy_of_Steering_wheel_angle_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip /content/drive/My\ Drive/project_dataset/dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/IMG/center_2016_12_01_13_37_09_903.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_13_636.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_16_370.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_16_470.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_16_671.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_19_107.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_20_123.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_22_458.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_23_776.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_24_585.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_24_787.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_31_769.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_33_685.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_34_291.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_34_695.jpg  
  inflating: data/IMG/center_2016_12_01_13_37_34_998.jpg  
  inf

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import cv2

import numpy as np

import csv

In [0]:
#import cv2
#import torch 
#import torch.utils.data as data
#import numpy as np
#import csv

samples = []

# Step 1 : Reading the images from csv file
with open('/content/data/driving_log.csv') as csvfile:
  
  reader = csv.reader(csvfile)
  next(reader,None)
  for line in reader:
    samples.append(line)

# Step 2 : Dividing the dataset for Training and Testing

# 80 percent of total dataset for Training
# 20 percent of total dataset for Testing
train_len = int(0.8*len(samples))
test_len = len(samples) - train_len
train_samples, testing_samples = torch.utils.data.random_split(samples, lengths=[train_len, test_len])

In [0]:
#import cv2
#import torch
#import torch.utils.data as data
#import numpy as np

# Step 3a: Define the transformations process, augmentation and parameters for dataloader 
# What is augmentation ?
# Augmentation refers to the process of generating new training data from a smaller data set.
# Why cropping is done ?
# -> cropping, basically, helps the model to focus only on the road by taking away the sky and other distracting stuff in the image.
# Why flipping is done ?
# -> flip images at random and change the sign of the predicted angle to simulate driving in the opposite direction

def augment(imgName, angle):
  name = 'data/IMG/' + imgName.split('/')[-1]
  current_image = cv2.imread(name)
  #cropping
  # why those numbers ?
  # ->
  current_image = current_image[65:-25, :, :]
  #random fliping
  # why rand < 0.5 ?
  #-> 
  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0  
  return current_image, angle

In [0]:
class Dataset(data.Dataset):

    def __init__(self, samples, transform=None):

        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):
      
        batch_samples = self.samples[index]
        
        steering_angle = float(batch_samples[3])
        
        center_img, steering_angle_center = augment(batch_samples[0], steering_angle)
        left_img, steering_angle_left = augment(batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right = augment(batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img = self.transform(left_img)
        right_img = self.transform(right_img)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)
      
    def __len__(self):
        return len(self.samples)

In [0]:
# Step 3b : Creating generator using the dataloader to parallasize the process.
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 4}

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

training_set = Dataset(train_samples, transformations)
training_generator = data.DataLoader(training_set, **params)

testing_set = Dataset(testing_samples, transformations)
testing_generator = data.DataLoader(testing_set, **params)

In [0]:
# Step4: Define the network
class NetworkDense(nn.Module):

    def __init__(self):
        super(NetworkDense, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64 * 2 * 33, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, input):  
        input = input.view(input.size(0), 3, 70, 320)
        output = self.conv_layers(input)
        print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output

In [0]:
model = NetworkDense()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

In [11]:
# Step6: Check the device and define function to move tensors to that device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('device is: ', device)

def toDevice(datas, device):
  
  imgs, angles = datas
  return imgs.float().to(device), angles.float().to(device)

device is:  cuda


In [12]:
# Step7: Train and validate network based on maximum epochs defined
max_epochs = 22
for epoch in range(max_epochs):
    model.to(device)
    
    # Training
    train_loss = 0
    model.train()
    for local_batch, (centers, lefts, rights) in enumerate(training_generator):
        # Transfer to GPU
        centers, lefts, rights = toDevice(centers, device), toDevice(lefts, device), toDevice(rights, device)
        
        # Model computations
        optimizer.zero_grad()
        datas = [centers, lefts, rights]        
        for data in datas:
            imgs, angles = data
#             print("training image: ", imgs.shape)
            outputs = model(imgs)
            loss = criterion(outputs, angles.unsqueeze(1))
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()
            
        if local_batch % 100 == 0:
            print('Loss: %.3f '
                % (train_loss/(local_batch+1)))

Streaming output truncated to the last 5000 lines.
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33])
torch.Size([32, 64, 2, 33

In [13]:
state = {
        'model': model.module if device == 'cuda' else model,
        }

torch.save(state, 'model.h5')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type NetworkDense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
